## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

In [50]:
import numpy as np
import pandas as pd
import string

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [51]:
A = np.random.randint(0, 1001, size=1_000_000)
A

def f1(A):
    acc, cnt = 0, 0
    for x in A:
        b = x + 100
        acc += b
        cnt += 1
    return acc / cnt

f1(A)

600.043022

In [52]:
%time f1(A) # на уровне 1 строчки

CPU times: total: 312 ms
Wall time: 324 ms


600.043022

In [53]:
%%timeit
f1(A)

292 ms ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
def f2(A):
    acc, cnt = 0, len(A)
    for x in A:
        acc += x
    return acc / cnt + 100

In [55]:
%%time
f2(A)

CPU times: total: 125 ms
Wall time: 118 ms


600.0430220000001

In [56]:
def f3(A):
    return np.mean(A) + 100

In [57]:
%%time
f3(A)

CPU times: total: 0 ns
Wall time: 1.99 ms


600.0430220000001

In [58]:
%load_ext line_profiler

In [59]:
%lprun -f f1 f1(A)


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [60]:
N = 2_000_000
df = pd.DataFrame(np.random.randn(N, 4), columns=[f"col{i}" for i in range(4)])
df["key"] = np.random.choice(list(string.ascii_letters.lower()), N, replace=True)
df.head(2)

,col0,col1,col2,col3,key
0,0.344286,-0.113528,1.501144,0.241932,s
1,0.247780,-1.297846,0.385017,0.357652,e


In [ ]:
def g1(df):
    mask = []
    for _, row in df.iterrows():
        if row["key"] in {"a", "b", "c", "d", "e"}:
            mask.append(True)
        else:
            mask.append(False)
    r = df[mask]
    return r

In [ ]:
%%time
g1(df)

In [ ]:
%lprun -f g1 g1(df.head(20_000))

In [ ]:
def g2(df):
    mask = df["key"].isin({"a", "b", "c", "d", "e"})
    return df[mask]

In [ ]:
%%time
g2(df)

## Лабораторная работа 3

In [ ]:
# !pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [ ]:
recipes = pd.read_csv('recipes_sample.csv')
reviews = pd.read_csv('reviews_sample.csv')
reviews['date'] = pd.to_datetime(reviews['date'])

In [ ]:
def mean_rating_A(df):
    total = 0
    count = 0
    for _, row in df.iterrows():
        if row['date'].year == 2010:
            total += row['rating']
            count += 1
    return total/count
mean_rating_A(reviews)

In [ ]:
reviews_2010 = reviews[reviews["date"].dt.year == 2010]

In [ ]:
def mean_rating_B(df):
    total = 0
    count = 0
    for _, row in df.iterrows():
        total += row["rating"]
        count += 1
    return total / count
mean_rating_B(reviews_2010)

In [ ]:
def mean_rating_C(df):
    return df['rating'].mean()
mean_rating_C(reviews_2010)

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер line_profiler. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

In [ ]:
!pip install line_profiler

In [ ]:
from line_profiler import LineProfiler
lp = LineProfiler()
lp_wrapper_A = lp(mean_rating_A)
lp_wrapper_B = lp(mean_rating_B)
lp_wrapper_C = lp(mean_rating_C)

lp_wrapper_A(reviews)
lp_wrapper_B(reviews_2010)
lp_wrapper_C(reviews_2010)
lp.print_stats()

In [ ]:
#оптимизируем функцию Б
def mean_rating_B2(df):
    total = reviews_2010["rating"].sum()
    count = reviews_2010["rating"].count()
    return total/count
mean_rating_B2(reviews_2010)

In [ ]:
lp.print_stats()

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [ ]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [ ]:
from collections import Counter

def get_word_reviews_count_optimized(df):
    words_counts = Counter()
    df_without_na = df.dropna(subset=["review"])

    def update_word_counts(row):
        words = set(row)
        words_counts.update(words)
    
    pattern = r"[^A-Za-z\s]"
    df_without_na['review'].str.replace(pattern, "").str.lower().str.split(" ").apply(update_word_counts)


    return dict(words_counts.items())

In [ ]:
lp_get_word_reviews_count = lp(get_word_reviews_count)
lp_get_word_reviews_count(reviews)
lp_get_word_reviews_count_optimized = lp(get_word_reviews_count_optimized)
lp_get_word_reviews_count_optimized(reviews)
lp.print_stats()
C:\Users\User\AppData\Loc

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [ ]:
#Без использования векторизованных операций и методов массивов numpy и без использования numba:
def MAPE_A(df):
    ratings = [r for r in df['rating'] if r != 0]
    mean_rating = sum(ratings) / len(ratings)
    abs_diff = 0
    for r in ratings:
        abs_diff += abs(r - mean_rating)
    return abs_diff / len(ratings) * 100
MAPE(reviews)

In [ ]:
#Без использования векторизованных операций и методов массивов numpy, но с использованием numba
import numba 

@numba.jit(nopython=True)
def MAPE(reviews):
    ratings = [r for r in reviews['rating'] if r != 0]
    mean_rating = sum(ratings) / len(ratings)
    abs_diff = 0
    for r in ratings:
        abs_diff += abs(r - mean_rating)
    return abs_diff / len(ratings) * 100
MAPE(reviews)

In [ ]:
import numpy as np

def MAPE_C(reviews):
    ratings = np.array([r for r in reviews['rating'] if r != 0])
    mean_rating = np.mean(ratings)
    abs_diff = np.sum(np.abs(ratings - mean_rating))
    return abs_diff / len(ratings) * 100
MAPE_C(reviews)